In [1]:
import torch

In [2]:
import Mydataloader

In [3]:
import Mymodel

In [4]:
import losses

In [5]:
path = '/root/Zero-DCE/Zero-DCE_code/data/DepthAndImage/'
imagedepthdataset = Mydataloader.ImageDepthLoader(path, path + "Image/", path + "Depth/")

Total training examples: 3000


In [6]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


epochs = 100
        
DCE_net = Mymodel.enhance_net().cuda()
DCE_net.apply(weights_init)

train_dataset = Mydataloader.ImageDepthLoader(path, path + "Image/", path + "Depth/", 2024)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)


optimizer = torch.optim.Adam(DCE_net.parameters(), lr=0.0001, weight_decay=0.0001)

DCE_net.train()

L_color = losses.L_color()
L_spa = losses.L_spa()
L_exp = losses.L_exp(16,0.5)
L_TV = losses.L_TV()
L_power = losses.L_power(0.8)
L_Depth = losses.L_Depth(0.8)
L_color_frac = losses.L_color_frac(11)
c = 1e-5

for epoch in range(epochs):
    print(f"------------------------{epoch} Epoch------------------------")
    for iteration, (input_img, depth) in enumerate(train_loader):
        input_img = input_img.cuda()
        depth = depth.unsqueeze(1).cuda()

        output_img_1, output_img, A  = DCE_net(input_img, depth)
        
        Loss_TV = 200 * L_TV(A)

        loss_spa = torch.mean(L_spa(output_img, input_img))

        loss_col = 15 * torch.mean(L_color(output_img))
        loss_col_frac = 15 * L_color_frac(input_img, output_img)
        
        loss_exp = 10*torch.mean(L_exp(output_img))
        loss_power = 5 * torch.mean(L_power(input_img, output_img))
        
        loss_depth = c * L_Depth(output_img, depth)
        
        # best_loss
        loss =  Loss_TV + loss_spa + loss_col + loss_exp + loss_power + loss_col_frac + loss_depth
        
torch.save(DCE_net.state_dict(), f"snapshots/Depth{c}{epoch + 1}.pth")
        

Total training examples: 2024
------------------------0 Epoch------------------------
torch.Size([8, 3, 256, 256])


RuntimeError: Output 0 of SplitBackward0 is a view and is being modified inplace. This view is the output of a function that returns multiple views. Such functions do not allow the output views to be modified inplace. You should replace the inplace operation by an out-of-place one.